# Dependencies

In [ ]:
!pip install azureml

In [ ]:
!pip install azureml.core

In [ ]:
!pip install azureml.pipeline

In [ ]:
!pip install azureml.widgets

# Pipeline setup

In [ ]:
from azureml.core import Workspace, Datastore

ws = Workspace.create(
   name = 'hackingjunction',
   subscription_id = 'a59b43cd-1c53-4a7e-b715-19b07f028dcf',
   resource_group = 'hacking',
   location = 'WestEurope',
   exist_ok = True)

In [ ]:
account_key = input()

In [ ]:
storage_account = "hackingjunctio1306266651"
container_name = "inputs"

In [ ]:
from azureml.core.runconfig import CondaDependencies, RunConfiguration

cd = CondaDependencies.create(pip_packages =['azureml-defaults', 'pandas', 'numpy', 'scipy', 'sklearn','azure-storage-blob==2.1.0'])

run_config = RunConfiguration(conda_dependencies = cd)
run_config.environment.docker.enabled = True
run_config.environment.spark.precache_packages = False

In [ ]:
def_blob_store = Datastore(ws, 'workspaceblobstore')

In [ ]:
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import PipelineData
from azureml.pipeline.steps.python_script_step import PythonScriptStep
import os
from azureml.core.compute import ComputeTarget

target = ws.compute_targets["JunctionCompute"]

input_ref = PipelineData(
    f"input",
    datastore = def_blob_store,
    output_name=f"input")

output_ref = PipelineData(
    f"output",
    datastore = def_blob_store,
    output_name=f"output")

train_step = PythonScriptStep(
    script_name = 'train.py',
    arguments = ['--input_path', input_ref,
                 '--output', output_ref,            
                 '--input_container', container_name,
                 '--output_container', 'outputs',
                 '--input_account', storage_account,
                 '--input_key', account_key],
    outputs = [output_ref, input_ref],
    compute_target = target,
    source_directory = './BehavioralAnalysis',
    allow_reuse = False,
    runconfig = run_config
)

pipeline_steps = [train_step]

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(workspace = ws, steps = pipeline_steps)

In [ ]:
from azureml.core.experiment import Experiment
from azureml.widgets import RunDetails

pipeline_run = Experiment(ws, 'Behavioral_Analysis_Experiment').submit(pipeline)

#pipeline_run.wait_for_completion(show_output = True)
RunDetails(pipeline_run).show()

# Deployment

In [ ]:
from azureml.pipeline.core import PublishedPipeline

pipelines = list(filter(lambda x: x._name == 'Behavioral_Analysis_Experiment', PublishedPipeline.list(ws)))
for pipeline in pipelines:
    pipeline.disable()

In [ ]:
pipeline = Pipeline(workspace = ws, steps = pipeline_steps)

published_pipeline = pipeline.publish(
    name="Behavioral_Analysis_Experiment",
    description="Retrains the model & computes behavioral tags for users",
    version=0.1)